In [1]:
# Data preprocessing

In [2]:
#1. Read tweets 
import csv
from nltk.tokenize import word_tokenize

disaster_file = "2012_Sandy_Hurricane-ontopic_offtopic.csv"

with open(disaster_file, 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    disaster_data = list(reader)
    disaster_header = reader.fieldnames

initial_datafile_len = len(disaster_data)
print "len of datafile:", initial_datafile_len
disaster_header = ['tweet id', 'tweet', 'label']
# remove spaces in header labels
for row in disaster_data:
    row['tweet'] = row[' tweet']
    del[row[' tweet']]
    row['label'] = row[' label']
    del[row[' label']]
assert( initial_datafile_len == len(disaster_data) )

len of datafile: 10008


In [23]:
# 2. Process tweets: Remove non-printable, ASCII characters 

import re

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

# v2
def tokenise(tweet):
    # Expand "'ve"
    tweet = re.sub(r"(?i)(\w)'ve\b", r'\1 have', tweet)
    # Separate punctuation from words
    tweet = re.sub(r'\b', ' ', tweet)
    # Remove isolated, single-character punctuation,
    # and articles (a, an, the)
    tweet = re.sub(r'(^|\s)([.;:]|[Aa]n|a|[Tt]he)($|\s)', r'\1\3', tweet)    
    # Split into non-empty strings
    return filter(bool, re.split(r'\s+', tweet))
 
def get_tokens(s, lowercase=False):
    tokens = tokens_re.findall(s) 
    #if lowercase:
    tokens = [token if emoticon_re.search(token) else token for token in tokens]
    return tokens

def remove_junk_symbols(tweet):
    print "tweet1", tweet
    tweet = tweet.split()
    
    articles = {'a', 'the'}
    non_readable = {'RT', '&', '&amp;', '@', ',', 'amp', ';', 'rt'}
    for i, token in enumerate(tweet):
        if (token in articles) or (token in non_readable):
            tweet.remove(token)
        elif token[0] == '@': # remove user mentioning
            tweet.remove(token)
            tweet.insert(i, 'username')
            print "token removed", token
        elif 'http' in token: 
            tweet.remove(token)
            tweet.insert(i, 'URL')
    
    print "tweet2", tweet
    tweet = (" ").join(tweet)
    print "tweet3", tweet
    # Expand "'ve"
    tweet = re.sub(r"(?i)(\w)'ve\b", r'\1 have', tweet)
    # Separate punctuation from words
    tweet = re.sub(r'\b', ' ', tweet)
    tweet = [token if emoticon_re.search(token) else token for token in tweet.split()] 
    print "tweet4", tweet
    print "______________________________________________________________________________"
    return tweet
    
def preprocess(tweet):
    #tweet = get_tokens(tweet) # v1 text mining
    #final_tweet = tokenise(tweet) #v2
    tweet = remove_junk_symbols(tweet)
    
    return tweet

In [24]:
import datetime
i = 0
preprocessed_disaster_data = list()
for row in disaster_data:
    preprocessed_tweet = preprocess(row['tweet'])
    if len(preprocessed_tweet) != 0:
        new_row = {'tweet id': row['tweet id'], 
                   'tweet': preprocessed_tweet,
                   'label': row['label']}
        preprocessed_disaster_data.append(new_row) 
    i += 1
    if i == 20:
        break
    
print "len(preprocessed_disaster_data):", len(preprocessed_disaster_data)

import datetime
now = datetime.datetime.now().strftime("%Y-%m-%d %H%M")

preprocessed_disaster_file = str(now) + "_Preprocessed_"+disaster_file
with open(preprocessed_disaster_file, 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=disaster_header)
    # show the header fields
    writer.writeheader()
    writer.writerows(preprocessed_disaster_data)

tweet1 I've got enough candles to supply a Mexican family;)!!!
tweet2 ["I've", 'got', 'enough', 'candles', 'to', 'supply', 'Mexican', 'family;)!!!']
tweet3 I've got enough candles to supply Mexican family;)!!!
tweet4 ['I', 'have', 'got', 'enough', 'candles', 'to', 'supply', 'Mexican', 'family', ';)!!!']
______________________________________________________________________________
tweet1 Sandy be soooo mad that she be shattering our doors and shiet #HurricaneSandy
tweet2 ['Sandy', 'be', 'soooo', 'mad', 'that', 'she', 'be', 'shattering', 'our', 'doors', 'and', 'shiet', '#HurricaneSandy']
tweet3 Sandy be soooo mad that she be shattering our doors and shiet #HurricaneSandy
tweet4 ['Sandy', 'be', 'soooo', 'mad', 'that', 'she', 'be', 'shattering', 'our', 'doors', 'and', 'shiet', '#', 'HurricaneSandy']
______________________________________________________________________________
tweet1 @ibexgirl thankfully Hurricane Waugh played it cool and waited this one out. Ready to go at any moment tho